# Little Lemon analysis and sales report
You have gained all the necessary skills and knowledge required to interact with the back-end MySQL database from your Python-based front-end application via the standard database client MySQL Connector/Python. 

Follow the instructions in the exercises given below and create a Jupyter notebook to produce a Little Lemon analysis and sales report. You need to submit this report for your peers to review.

## Task 1:
Complete the following steps to establish a connection pool:

- To create a connection pool, import MySQLConnectionPool class from MySQL Connector/Python.

- To find the information on the error, import the Error class from MySQL Connector/Python.

- Define your database configurations as a Python dictionary object called dbconfig.

- Establish a connection pool [pool_name = pool_b] with two connections. 

- Implement error handling using a try-except block in case the connection fails. 

In [1]:
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

dbconfig={"database":"littlelemondb", "user":"admin0", "password":"080011"}

try:
    pool = MySQLConnectionPool(pool_name='pool_b', pool_size=2, **dbconfig)
    print("The connection pool is created with a name: ", pool.pool_name)
    print("The pool size is:", pool.pool_size)
except Error as er:
    print("Error code:", er.errno)
    print("Error message:", er.msg)

The connection pool is created with a name:  pool_b
The pool size is: 2


## Task 2:
Three guests are trying to book dinner slots simultaneously. Get the connections from pool_b and insert the following data in the Bookings table:

- TIP1: You need to add a connection to connect the third guest.
- TIP2: The pool size is two. However, you have three connected users. You can only return two connections. Returning a third connection will raise a PoolError. Use try-except to print the error message. 

In [2]:
new_bookings = {"TableNo" : [8, 5, 12], "CustomerID" : [1, 4, 5], 
                "BookingDate" : ["2023-5-19", "2023-5-20", "2023-5-21"]}
n_bookings = len(new_bookings["CustomerID"])

print("{} new bookings ready to add to the Bookings Table".format(n_bookings))

3 new bookings ready to add to the Bookings Table


In [3]:
sql_insert = """
INSERT INTO Bookings (TableNo, CustomerID, BookingDate)
VALUES
({}, {}, "{}")
""".format(new_bookings["TableNo"][0], new_bookings["CustomerID"][0], new_bookings["BookingDate"][0])
print(sql_insert)


INSERT INTO Bookings (TableNo, CustomerID, BookingDate)
VALUES
(8, 1, "2023-5-19")



In [4]:
import mysql.connector as connector

# create a list to store connections in pool
n_connection = ["connect%s" % x for x in range(n_bookings)] 

for i in range(n_bookings):

    table = new_bookings["TableNo"][i]
    book_date = new_bookings["BookingDate"][i]
    cust_id = new_bookings["CustomerID"][i]

    insert_sql = """
    INSERT INTO Bookings (TableNo, CustomerID, BookingDate)
    VALUES
    ({}, {}, "{}");
    """.format(table, cust_id, book_date)

    try:
        n_connection[i] = pool.get_connection()
        print("\n[CustomerID:{}] is connected.".format(cust_id))

    except Error as er:
        print("\n", er)
        print("Adding new connection in the pool.")
        connection = connector.connect(user='root', password='080011')
        pool.add_connection(cnx=connection)
        print("A new connection is added in the pool.\n")
        n_connection[i] = pool.get_connection()
        print("[CustomerID:{}] is connected.".format(cust_id))    

    finally:
        cursor = n_connection[i].cursor()
        cursor.execute(insert_sql)
        n_connection[i].commit()
        print("Insert a new booking of [CustomerID:{}], and the information are as follows:".format(cust_id))
        print("\tTable Number: {}".format(table))
        print("\tBooking Date: {}".format(book_date))       


[CustomerID:1] is connected.
Insert a new booking of [CustomerID:1], and the information are as follows:
	Table Number: 8
	Booking Date: 2023-5-19

[CustomerID:4] is connected.
Insert a new booking of [CustomerID:4], and the information are as follows:
	Table Number: 5
	Booking Date: 2023-5-20

 Failed getting connection; pool exhausted
Adding new connection in the pool.
A new connection is added in the pool.

[CustomerID:5] is connected.
Insert a new booking of [CustomerID:5], and the information are as follows:
	Table Number: 12
	Booking Date: 2023-5-21


In [5]:
n_connection

In [6]:
# return connection to pool
for i in range(len(n_connection)):
    try:
        n_connection[i].close()
        print("connection {} already closed.".format(i+1))
    except Error as er:
        print(er)

connection 1 already closed.
connection 2 already closed.
Failed adding connection; queue is full


## Task 3:
Create a report containing the following information:

  - The name and EmployeeID of the Little Lemon manager.

  - The name and role of the employee who receives the highest salary.

  - Guests who booked in May.

  - The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.

In [7]:
# get the connection
connection = pool.get_connection()
if connection.is_connected():
    cursor = connection.cursor()
    print("cursor object is successfully created.")
else:
    print("fail to create a cursor object.")

cursor object is successfully created.


In [12]:
print("Reoprt of Little Lemon")

# Part1
print("\nPart 1--The name and EmployeeID of the Little Lemon manager.")
part1_report = """
SELECT Employees.EmployeeID, Employees.Name, Employees.Role
FROM Employees
WHERE Employees.Role = 'Manager';
"""
cursor.execute(part1_report)
dataset_part1 = cursor.fetchall()
print(cursor.column_names)
for data in dataset_part1:
       print(data)


# Part2
print("\nPart 2--The name and role of the employee who receives the highest salary.")
part2_report = """
SELECT Employees.EmployeeID, Employees.Name, Employees.Role,
       CAST(SUBSTRING(REPLACE(Employees.AnnualSalary, ',', ''), 2) AS UNSIGNED) AS 'Annual_Salary'
FROM Employees
ORDER BY 'Annual_Salary' DESC
LIMIT 1;
"""
cursor.execute(part2_report)
dataset_part2 = cursor.fetchall()
print(cursor.column_names)
for data in dataset_part2:
       print(data)


# Part3
print("\nPart 3--Guest who booked in May.")
part3_report = """
SELECT Bookings.BookingID, Bookings.BookingDate, 
       CONCAT(Customers.FirstName, ' ', Customers.LastName) AS GuestName,
       Customers.ContactNumber, Customers.Email
FROM Bookings
JOIN Customers ON Customers.CustomerID = Bookings.CustomerID
WHERE MONTH(BookingDate) = 5;
"""
cursor.execute(part3_report)
dataset_part3 = cursor.fetchall()
print(cursor.column_names)
for data in dataset_part3:
       print(data)


# Part4
import datetime as dt
print("\nPart 4--The full name and BookingID of all guests' priority.")
part4_report = """
SELECT CONCAT(Customers.FirstName, ' ', Customers.LastName) AS GuestName,
       Bookings.BookingID, Bookings.BookingDate,
       CASE
           WHEN Bookings.EmployeeID = 1 THEN 'Critical'
           WHEN Bookings.EmployeeID = 2 THEN 'High'
           WHEN Bookings.EmployeeID = 3 THEN 'Medium'
           ELSE 'Low'
       END AS Priority
FROM Bookings
LEFT JOIN Customers ON Customers.CustomerID = Bookings.CustomerID
LEFT JOIN Employees ON Employees.EmployeeID=Bookings.EmployeeID
ORDER BY Bookings.BookingDate ASC;
"""
cursor.execute(part4_report)
dataset_part4 = cursor.fetchall()
print(cursor.column_names)
for data in dataset_part4:
       print(data)

Reoprt of Little Lemon

Part 1--The name and EmployeeID of the Little Lemon manager.
('EmployeeID', 'Name', 'Role')
(1, 'Mario Gollini', 'Manager')

Part 2--The name and role of the employee who receives the highest salary.
('EmployeeID', 'Name', 'Role', 'Annual_Salary')
(1, 'Mario Gollini', 'Manager', 70000)

Part 3--Guest who booked in May.
('BookingID', 'BookingDate', 'GuestName', 'ContactNumber', 'Email')
(1, datetime.date(2022, 5, 19), 'Anna Iversen', 44627187, 'annaiversen@163.com')
(23, datetime.date(2023, 5, 19), 'Anna Iversen', 44627187, 'annaiversen@163.com')
(2, datetime.date(2022, 5, 20), 'Joakim Iversen', 74237178, 'joakimiii@personal.com')
(24, datetime.date(2023, 5, 20), 'Marcos Romero', 17732934, 'macosromero@softcorp.com')
(6, datetime.date(2022, 5, 20), 'Hiroki Yamane', 28832006, 'hirokiyamane@homeuse.com')
(25, datetime.date(2023, 5, 21), 'Hiroki Yamane', 28832006, 'hirokiyamane@homeuse.com')

Part 4--The full name and BookingID of all guests' priority.
('GuestName',

In [13]:
# return the connection to the pool
if connection.is_connected():
    connection.close()
    print("MySQL connection is closed.")
else:
    print("Connection is already closed")

MySQL connection is closed.
